In [4]:
import open3d
import numpy as np
import os
import utils.fread as fread
import utils.FCGF as FCGF
import utils.registration as registration

In [2]:
sequence_dir = "../localization-data/data/features/exp_5/trial_1/0.03/subject-1/01"
sequence_ts = fread.get_timstamps(sequence_dir)

In [3]:
source = FCGF.get_features(os.path.join(sequence_dir, f"{sequence_ts[0]}.secondary.npz"), pcd_only=True)

AttributeError: module 'open3d.cpu.pybind.geometry' has no attribute 'estimate_normals'

In [5]:
data = np.load(os.path.join(sequence_dir, f"{sequence_ts[0]}.secondary.npz"))

In [6]:
pcd = open3d.t.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(data["keypts"])

AttributeError: 'open3d.cpu.pybind.t.geometry.PointCloud' object has no attribute 'points'

In [ ]:
    keypts = open3d.geometry.PointCloud()
    keypts.points = open3d.utility.Vector3dVector(data["keypts"])
    
    open3d.geometry.estimate_normals(keypts)
    
    if pcd_only:
        return keypts
    scores = data["scores"]
    features = open3d.registration.Feature()
    features.data = data["features"].T
    return keypts, features, scores